In [19]:
import numpy as np
import pandas as pd
import random
from bs4 import BeautifulSoup
import seaborn as sns
import matplotlib.pyplot as plt
import lxml
import requests
import re
import time
from lxml import etree
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from re import match
from fake_useragent import UserAgent
import fake_useragent
import scrapy

In [20]:
ua = UserAgent()

In [24]:
cd ..

C:\Users\finnr\Downloads\Zhao McIntire Research


In [832]:
import socketserver

In [25]:
import scrapeimports
from scrapeimports import get_rot_prox

In [826]:
ip_ads = get_rot_prox()
random.choice(ip_ads)

'154.202.113.197'

### Verify If Accessible

In [48]:
url = f'https://www.baseball-reference.com/teams/NYM/2021.shtml'
page = requests.get(url, proxies={'http':random.choice(ip_ads)})

In [ ]:
def getPlayers(season_yr):
    
    """
    GRABS ALL PLAYERS IN AN MLB SEASON
    
    input: int season_yr (basketball season year. eg 2021 represents 2020-2021 NBA season)
    output: list[str] of NBA Players (format: 'Lastname,Firstname')
    """
    
    
    #URL for all players in the season
    url = f'https://www.basketball-reference.com/leagues/NBA_{season_yr}_totals.html#totals_stats'
    page = requests.get(url, proxies={'http':random.choice(ip_ads)})
    
    #Grab the list from the website
    soup = BeautifulSoup(page.text, 'lxml')
    all_people = soup.find_all('tr',class_='full_table')
    
    #Go through list and collect each player
    nba_players = []
    for person in all_people:
        nba_players.append(person.find('td')['csk'])
        
    return nba_players

### TeamCodes

In [66]:
#URL for all players in the season
url = f'https://www.baseball-reference.com/leagues/majors/2021-standings.shtml'
page = requests.get(url, proxies={'http':random.choice(ip_ads)})

#Grab the list from the website
soup = BeautifulSoup(page.text, 'lxml')

In [172]:
def teaminits(x):
    return x.replace('/teams/','').replace('/2021.shtml','')

In [173]:
teams = [[teaminits(x.find('th').find('a')['href']) for x in soup.find_all('table')[n].find('tbody').find_all('tr')] for n in range(6)]

In [174]:
teamcodes = [t for sublist in teams for t in sublist]

### TeamPage

In [219]:
from bs4 import Comment

In [966]:
all_soup = list()

In [967]:
total_coach_info = pd.DataFrame(columns = [0,1,2,3,4,5,"Coach","URL","TM","Season"])

In [968]:
total_player_info = pd.DataFrame(columns = range(28))
total_player_info[["Player","URL","TM","Season"]] = 0

In [946]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--no-default-browser-check')
options.add_argument('--disable-gpu')
options.add_argument('--disable-extensions')
options.add_argument('--disable-default-apps')

ser = Service("C:\\Users\\finnr\\Downloads\\Zhao McIntire Research\\chromedriver_win32\\chromedriver.exe")

In [1007]:
driver = webdriver.Chrome(ChromeDriverManager().install(),service=ser,options=options)

C:\Users\finnr\AppData\Local\Temp\ipykernel_17812\2294712664.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(),service=ser,options=options)


In [1009]:
def get_people_info_sel(teamcode,yr=2021):
    
    #starting information for link
    url = f'https://www.baseball-reference.com/teams/{teamcode}/{yr}.shtml'
    
    while (yr != 1998):
        
        #Access Page
        headers = {"User-Agent": ua.random,
           "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
           "Accept-Language": "en-US,en;q=0.7",
           "Accept-Encoding": "gzip, deflate",
           "Referer": "https://www.google.com/",
           "DNT": "1", "Connection": "close", "Upgrade-Insecure-Requests": "1"}
        
        #Get Information From Page
        global page
        global soup
        
        
        driver.get(url)
        
        src = driver.page_source
        time.sleep(3.1)
        soup = BeautifulSoup(src, 'lxml')
        all_soup.append(soup)

        url = 'https://www.baseball-reference.com' + soup.find('a',class_='button2 prev')['href']


        #Coach Convert Commented Table to Soup
        coach = BeautifulSoup(
            [i for i in soup.find_all(string=lambda text: isinstance(text, Comment)) if 'id="coaches"' in i][0]).find('tbody')

        #Grab Coach Names
        coaching_staff = [x.get_text() for x in coach.find_all('th')]

        #Grab Coach Info Chart
        coach_info = np.array([x.get_text() for x in coach.find_all('td')]).reshape(len(coaching_staff),-1)

        #Get Links for Each Coach
        hrefs_coach = [x.find('a') for x in coach.find_all('th')]
        hrefs_coach = [None if x is None else x['href'] for x in hrefs_coach]

        # Create Dataframe for Coaching Information from Team 'tcode' during season 'yr'
        # {0:'Age',1:'Country',2:'DoB',3:'Role',4:'Start Date', 5: 'End Date'}
        coach_info = pd.DataFrame(coach_info)
        coach_info['Coach'] = coaching_staff
        coach_info['URL'] = hrefs_coach
        coach_info['TM'] = teamcode
        coach_info["Season"] = yr

        global total_coach_info
        total_coach_info = pd.concat([total_coach_info,coach_info],ignore_index=True)


        #Player Convert Commented Table to Soup
        player = BeautifulSoup(
            [i for i in soup.find_all(string=lambda text: isinstance(text, Comment)) if 'id="appearances"' in i][0]).find('tbody')

        #Get Player Names
        players = [x.get_text() for x in player.find_all('th')]

        #Get Player Info Chart
        player_info = np.array([x.get_text() for x in player.find_all('td')]).reshape(len(players),-1)

        #Get Links for Each Player
        hrefs_players = [x.find('a') for x in player.find_all('th')]
        hrefs_players = [None if x is None else x['href'] for x in hrefs_players]

        # Create Dataframe for Player Information from Team 'tcode' during season 'yr'
        player_info = pd.DataFrame(player_info)
        player_info['Player'] = players
        player_info['URL'] = hrefs_players
        player_info['TM'] = teamcode
        player_info["Season"] = yr

        global total_player_info
        total_player_info = pd.concat([total_player_info,player_info],ignore_index=True)

        yr -= 1

In [1010]:
def big_soup(soups,teamcode):
    # prettify the soupfied html elements in the list
    links_prettified = [s.prettify() for s in soups]

    # add the custom word to standout, I chose BREAKHERE
    list_with_word = [m+'BREAKHERE' for m in links_prettified]

    # make one giant string, using the .join trick list_as_one = "".join(list_with_word)
    # save the giant string to .txt with open('soup_list.txt', 'w') as file:   file.write(list_as_one)

    # In order to know the team and season:
    # title = soup.find_all('div',{'data-template':'Partials/Teams/Summary'})[0].find_all('span')
    # title[0].get_text() +' ' + title[1].get_text()
    
    with open(f'baseballsoup{teamcode}.txt', 'w', encoding='utf-8') as f:
        for soup in list_with_word:
            f.write(soup)
    

In [1015]:
for n in np.arange(len(teamcodes)):
    t = teamcodes[n]

    print("Team: ", t)
    
    #Create Copy Before It Gets Manipulated
    temp_total_player_info = total_player_info.copy()
    temp_total_coach_info = total_coach_info.copy()

    #Reset the Soups
    all_soups = list()

    #Get Info
    get_people_info_sel(t)

    #Put Soups into .txt File
    big_soup(all_soup,t)

Team:  TOR
Team:  BAL
Team:  CHW
Team:  CLE
Team:  DET
Team:  KCR
Team:  MIN
Team:  HOU
Team:  SEA
Team:  OAK
Team:  LAA
Team:  TEX
Team:  ATL
Team:  PHI
Team:  NYM
Team:  MIA
Team:  WSN
Team:  MIL
Team:  STL
Team:  CIN
Team:  CHC
Team:  PIT
Team:  SFG
Team:  LAD
Team:  SDP
Team:  COL
Team:  ARI


In [1022]:
total_player_info.to_csv('C:\\Users\\finnr\\Downloads\\Zhao McIntire Research\\Baseball\\teamrosters.csv')
total_coach_info.to_csv('C:\\Users\\finnr\\Downloads\\Zhao McIntire Research\\Baseball\\coaches.csv')